In [1]:
import pandas as pd
import numpy as np

In [2]:
app_launch = pd.read_csv('../input/app_launch_log.txt',header=None,sep='\t',names=['user_id','launch_day'])
user_activity = pd.read_csv('../input/user_activity_log.txt',header=None,sep='\t',names=['user_id','activity_day','page','video_id','author_id','action_type'])
user_register = pd.read_csv('../input/user_register_log.txt',header=None,sep='\t',names=['user_id','register_day','register_type','device_type'])
video_create = pd.read_csv('../input/video_create_log.txt',header=None,sep='\t',names=['user_id','create_day'])

In [3]:
data1_user=user_register[(user_register['register_day'] >= 1) & (user_register['register_day'] <= 16)]
data2_user=user_register[(user_register['register_day'] >= 1) & (user_register['register_day'] <= 23)]

In [4]:
print(data1_user.shape)
print(len(np.unique(data1_user['user_id'])))
print(data2_user.shape)
print(len(np.unique(data2_user['user_id'])))

(23848, 4)
23848
(37446, 4)
37446


In [5]:
temp_create = video_create[(video_create['create_day'] >= 1) & (video_create['create_day'] <= 16)]
temp_launch = app_launch[(app_launch['launch_day'] >= 1) & (app_launch['launch_day'] <= 16)]
temp_activity = user_activity[(user_activity['activity_day'] >= 1) & (user_activity['activity_day'] <= 16)]
data2_temp_create = video_create[(video_create['create_day'] >= 8) & (video_create['create_day'] <= 23)]
data2_temp_launch = app_launch[(app_launch['launch_day'] >= 8) & (app_launch['launch_day'] <= 23)]
data2_temp_activity = user_activity[(user_activity['activity_day'] >= 8) & (user_activity['activity_day'] <= 23)]

In [6]:
print(len(np.unique(pd.concat([temp_create['user_id'],temp_launch['user_id'],temp_activity['user_id']]))))
print(len(np.unique(pd.concat([data2_temp_create['user_id'],data2_temp_launch['user_id'],data2_temp_activity['user_id']]))))

23848
33992


In [10]:
# 极差
def ptp(column):
    return max(column) - min(column)
# 方差除以评率
def var_divide_count(column):
    return np.var(column) / len(column)

def last_one(column):
    if len(column) < 1:
        return column
    return column.iloc[-1]
def last_one_minus_reg():
    return

def q25(column):
    return column.quantile(.25)


def q75(column):
    return column.quantile(.75)


def kurt(column):
    return column.kurt()

In [12]:
# APP 启动次数，平均次数，方差，启动天数，最大值，最小值，连续几天启动总次数，平均次数，某一天启动次数
# agg要使用的函数
func = ['min', 'max', 'mean', 'median', 'skew', 'std', 'mad', 'count', ptp, np.var, var_divide_count, last_one, q25, q75, kurt]
app = temp_launch.groupby('user_id')['launch_day'].agg(func).reset_index()
app = app.rename(columns={'min': 'app_min', 'max': 'app_max', 'mean': 'app_mean', 'median': 'app_median',
             'skew': 'app_skew', 'std': 'app_std', 'mad': 'app_mad', 'count': 'app_count',
             'ptp': 'app_ptp', 'var': 'app_var', 'var_divide_count': 'app_var_divide_count',
             'last_one': 'app_last_one', 'q25': 'app_q25', 'q75': 'app_q75', 'kurt': 'app_kurt'})

In [13]:
app.head()

,user_id,app_min,app_max,app_mean,app_median,app_skew,app_std,app_mad,app_count,app_ptp,app_var,app_var_divide_count,app_last_one,app_q25,app_q75,app_kurt
0,16,13,15,14.00,14.0,0.000000,1.000000,0.666667,3,2,1.000000,0.222222,15,13.50,14.50,NaN
1,98,16,16,16.00,16.0,NaN,NaN,0.000000,1,0,NaN,0.000000,16,16.00,16.00,NaN
2,105,12,16,14.25,14.5,-0.752837,1.707825,1.250000,4,4,2.916667,0.546875,16,13.50,15.25,0.342857
3,218,11,16,13.50,13.5,0.000000,1.870829,1.500000,6,5,3.500000,0.486111,11,12.25,14.75,-1.200000
4,225,7,7,7.00,7.0,NaN,NaN,0.000000,1,0,NaN,0.000000,7,7.00,7.00,NaN


In [26]:
launch_day_diff = pd.concat([app_launch['user_id'], app_launch.groupby(['user_id']).diff().rename({'launch_day': 'launch_day_diff'}, axis=1)], axis=1)

In [44]:
launch_total_count = app_launch[['user_id']].groupby(['user_id']).size().rename('launch_total_count').reset_index()
launch_rest_day = (end_day - app_launch.groupby(['user_id'])['launch_day'].max() + 1).rename('launch_rest_day').reset_index()

In [50]:
    launch_rest_day['launch_average']=launch_total_count['launch_total_count'].div(launch_rest_day['launch_rest_day'])

In [46]:
launch_average=launch_total_count['launch_total_count']/launch_rest_day['launch_rest_day']

In [47]:
launch_average.head()

0     1.125000
1     0.142857
2     0.066667
3    15.000000
4     4.000000
dtype: float64

In [48]:
launch_total_count.head()

,user_id,launch_total_count
0,16,9
1,30,1
2,98,1
3,105,15
4,176,4


In [51]:
launch_rest_day.head()

,user_id,launch_rest_day,launch_average
0,16,8,1.125000
1,30,7,0.142857
2,98,15,0.066667
3,105,1,15.000000
4,176,1,4.000000


In [29]:
app_launch.groupby(['user_id']).diff().rename({'launch_day': 'launch_day_diff'}, axis=1)

,launch_day_diff
0,NaN
1,NaN
2,5.0
3,2.0
4,1.0
5,6.0
6,6.0
7,2.0
8,2.0
9,NaN


In [27]:
launch_day_diff.head()

,user_id,launch_day_diff
0,383135,NaN
1,330986,NaN
2,330986,5.0
3,330986,2.0
4,330986,1.0


In [6]:
userPre = user_activity[user_activity['activity_day']>=24]
sub = userPre[['user_id']].drop_duplicates()
sub.to_csv("../submission/baseline0611.csv",encoding='utf-8',index=None,header=None)

In [31]:

df1 = pd.DataFrame({'A':[1,1,2,2],'B':[2,2,6,8],'C':[8,7,6,5]})
df2 = pd.DataFrame({'A':[3,1,6,9,56],'B':[4,2,9,10,76],'D':[1,2,3,4,25]})
#df = df1.merge(df2,how='outer',on=['A','B'],left_index=True)
#t = df1.groupby('A').agg({'C':'min'})

#t = df1.groupby(['A'])['C'].min().rename('T').reset_index()


In [36]:
t = df1.groupby('A').agg({'C':'min'})

In [37]:
t

,C
A,
1,7
2,5


In [38]:
df1

,A,B,C
0,1,2,8
1,1,2,7
2,2,6,6
3,2,8,5


In [41]:
t = df1.groupby('A').agg({'A':'mean'})